In [1]:
import numpy as np
n_features = 20
term_pred = 'long_term'
data_file_name = 'data_after_y10_origin_at_10_top_20_var'
training_id_file_name = 'all_training_ID_outerloop_cohort_10_26'
endpt = 16;
eval_times = 365.25*np.r_[np.arange(2, endpt+1, 2)]
l2_final=0.1
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'


In [2]:
# %pip install tensorflow
# %pip install lifelines

#!y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [3]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [4]:
# snippets_dir = work_dir+ '/code/snippet'
snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'

import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import cox_nnet
from cox import *


In [5]:
load_dir = work_dir+ '/csv_files'

# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [6]:
data_full

,ID,time,event,C09SMKYR,C02AVGDI,C02AVGSY,C02ARMCI,CL1LDL,CL1NTRIG,C08DIBAG_YEAR,...,C08CHOAG_YEAR,C20CHES1,C03ED,DPVAE,C20HIPS1,CL1HDL,C37DISTU,MMLAD,C20WGT,BSA
0,100016012504,5258.5,0,12,72.0,114.0,33.0,119.000000,147.0,0,...,0,95.5,12,0.450000,105.5,38.0,2,3.970000,180.0,1.970000
1,100023004268,5560.5,0,8,64.0,94.0,28.0,105.000000,89.0,0,...,0,94.0,16,0.600000,100.0,43.0,1,3.540000,169.5,1.970000
2,100033323702,5381.5,0,0,73.0,105.0,30.0,160.000000,110.0,0,...,0,93.5,19,0.490000,101.5,52.0,2,3.810000,169.0,1.900000
3,100056526386,5511.5,0,0,76.0,118.0,37.0,137.000000,91.0,0,...,0,112.5,16,0.590000,106.0,44.0,1,4.390000,219.5,2.150000
4,100061300991,5403.5,0,11,72.0,112.0,33.0,108.000000,260.0,0,...,0,102.0,14,0.560000,103.0,40.0,1,3.880000,216.5,2.210000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,416761219907,5439.5,0,0,63.0,98.0,25.0,150.000000,84.0,0,...,5,82.0,18,0.370000,98.5,57.0,1,3.430000,120.0,1.940000
4196,416771521620,5586.5,0,0,82.0,111.0,36.0,120.000000,38.0,0,...,0,102.0,15,0.830000,106.0,76.0,2,3.130000,184.0,1.630000
4197,416783315386,5563.5,0,3,64.0,109.0,25.0,90.000000,47.0,0,...,0,87.5,16,0.588047,96.5,58.0,1,3.415775,126.0,1.775142
4198,416796224310,5452.5,0,12,69.0,108.0,39.0,100.000000,40.0,0,...,0,111.0,13,0.790000,122.5,55.0,2,4.640000,229.0,2.120000


In [7]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/'+training_id_file_name+'.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [8]:
data_full

,ID,event,time,0,1,2,3,4,5,6,...,10,11,12,13,14,15,16,17,18,19
0,100016012504,0,5258.5,1.337183,0.289697,0.556814,0.352765,0.338175,0.964529,-0.088134,...,-0.20046,-0.002890,-0.996914,-0.935661,0.191852,-1.106289,0.474830,0.975881,0.314146,0.561146
1,100023004268,0,5560.5,0.664185,-0.511559,-1.194017,-0.660105,-0.108371,0.131610,-0.088134,...,-0.20046,-0.139066,0.657061,0.057745,-0.282004,-0.747398,-0.953742,0.036538,0.054760,0.561146
2,100033323702,0,5381.5,-0.681811,0.389854,-0.231060,-0.254957,1.645918,0.433184,-0.088134,...,-0.20046,-0.184458,1.897542,-0.670752,-0.152770,-0.101392,0.474830,0.626358,0.042409,0.250807
3,100056526386,0,5511.5,-0.681811,0.690325,0.906981,1.163060,0.912306,0.160331,-0.088134,...,-0.20046,1.540440,0.657061,-0.008482,0.234930,-0.675619,-0.953742,1.893379,1.289929,1.359161
4,100061300991,0,5403.5,1.168934,0.289697,0.381731,0.352765,-0.012683,2.587284,-0.088134,...,-0.20046,0.587207,-0.169926,-0.207163,-0.023537,-0.962733,-0.953742,0.779274,1.215819,1.625165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,416761219907,0,5439.5,-0.681811,-0.611716,-0.843851,-1.267826,1.326957,0.059807,-0.088134,...,2.20231,-1.228476,1.484049,-1.465477,-0.411237,0.257500,-0.953742,-0.203760,-1.168057,0.428144
4196,416771521620,0,5586.5,-0.681811,1.291267,0.294190,0.960486,0.370071,-0.600784,-0.088134,...,-0.20046,0.587207,0.243568,1.580966,0.234930,1.621288,0.474830,-0.859115,0.412959,-0.946214
4197,416783315386,0,5563.5,-0.177063,-0.511559,0.119107,-1.267826,-0.586814,-0.471538,-0.088134,...,-0.20046,-0.729163,0.657061,-0.021416,-0.583548,0.329278,-0.953742,-0.234833,-1.019837,-0.302741
4198,416796224310,0,5452.5,1.337183,-0.010774,0.031565,1.568208,-0.267852,-0.572063,-0.088134,...,-0.20046,1.404264,-0.583420,1.316058,1.656498,0.113943,0.474830,2.439509,1.524611,1.226158


In [12]:
# Outerloop:
for fold in range(25):

    #fold = 24

    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']


    model_params = dict(node_map = None, input_split = None)
    search_params = dict(method = "nesterov"
                         , learning_rate=0.0001
                         , momentum=0.9
                         , max_iter=200000
                         , stop_threshold=0.995
                         , patience=1000
                         , patience_incr=2
                         , rand_seed = 9114
                         , eval_step=23
                         , lr_decay = 0.9
                         , lr_growth = 1.0)

    # #train final model
    L2_reg = 0.1
    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values
    model_params = dict(node_map = None, input_split = None, L2_reg=0.1)
    cox_nnet_model, cox_nnet_cost_iter = cox_nnet.trainCoxMlp(featurespace_train
                                                              , train_df.time.values
                                                              , train_df.event.values
                                                              , model_params
                                                              , search_params
                                                              , verbose=True)
    cox_nnet_theta_train = cox_nnet_model.predictNewData(featurespace_train)
    cox_nnet_theta_test = cox_nnet_model.predictNewData(featurespace_test)

    #discrimination on train, test sets
    c_train = concordance_index(train_df.time,-cox_nnet_theta_train, train_df.event)
    print('C on training set:' + str(c_train))
    c_test = concordance_index(test_df.time,-cox_nnet_theta_test, test_df.event)
    print('C on test set:' + str(c_test))

    predicted_risk = cox_nnet_theta_test
    predicted_risk = pd.DataFrame(data=predicted_risk)
    times, H0 = cox_basehaz(cox_nnet_theta_train.astype('float32').flatten(), train_df.time.values, train_df.event.values.astype(bool))
    trained_time = pd.DataFrame(data = times)
    pred_prob_surv = cox_pred_surv(cox_nnet_theta_test.astype('float32').flatten(), H0)
    pred_prob_surv = pd.DataFrame(data=pred_prob_surv)


    savedir = os.path.join(work_dir,'csv_files/cox_nnet/'+str(n_features)+'_features/'+term_pred)
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1

    predicted_risk.to_csv(savedir+'/lp_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    pred_prob_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    trained_time.to_csv(savedir+'/trained_time_fold_'+str(actual_fold)+'.csv', index = None, header = True)




FOLD 5:

Using nesterov accelerated gradient
training model
cost: 1.162884, iteration: 0
cost: 1.150104, iteration: 46
cost: 1.142628, iteration: 69
cost: 1.135204, iteration: 92
cost: 1.127848, iteration: 115
cost: 1.120559, iteration: 138
cost: 1.113337, iteration: 161
cost: 1.106181, iteration: 184
cost: 1.099091, iteration: 207
cost: 1.092066, iteration: 230
cost: 1.085106, iteration: 253
cost: 1.078210, iteration: 276
cost: 1.071377, iteration: 299
cost: 1.064606, iteration: 322
cost: 1.057898, iteration: 345
cost: 1.051252, iteration: 368
cost: 1.044667, iteration: 391
cost: 1.038142, iteration: 414
cost: 1.031677, iteration: 437
cost: 1.025271, iteration: 460
cost: 1.018925, iteration: 483
cost: 1.012636, iteration: 506
cost: 1.006405, iteration: 529
cost: 1.000232, iteration: 552
cost: 0.994115, iteration: 575
cost: 0.988055, iteration: 598
cost: 0.982050, iteration: 621
cost: 0.976100, iteration: 644
cost: 0.970205, iteration: 667
cost: 0.964365, iteration: 690
cost: 0.958578